## Final Project Submission

### GROUP 4

Please fill out:
* Students name: **Beth Nyambura, Benson Ouma, Harrison Karime, Rahab Gachie**
* Student pace: **part time**
* Scheduled project review date/time: **29th March 2025, 11:59PM**
* Instructor name: **Brian Chacha**
* Blog post URL:


In [ ]:
# Your code here - remember to use markdown cells for comments as well!

#  Movie Studio Strategy: Data-Driven Decisions for Box Office Success

##  Introduction

In an increasingly competitive content landscape, companies are investing heavily in original video production. Our company is launching a new movie studio and aims to enter this space strategically. However, lacking prior experience in film production, we must rely on data to identify what makes a movie successful.

This project leverages real-world movie data to explore what types of films perform best at the box office. Through structured analysis, we aim to generate insights that can inform content creation, budgeting, casting, and release strategies.

---
##  Business Understanding

###  Business Objective

Our primary business question is:

> **What types of films should our company produce to maximize success at the box office?**

To answer this, we explore several supporting questions:

1. **What are the current trends by genre in the box office?**  
   - Which genres dominate the market?
   - Which ones generate the highest gross and ROI?

2. **What budget ranges are the most profitable?**  
   - How can we optimize spending without compromising success?

3. **Which directors and actors consistently attract larger audiences?**  
   - Who are the “bankable names” we should work with?

4. **What is the average length (runtime) of films that perform well?**  
   - Is there an ideal duration for maximizing audience engagement?

5. **When is the best time to release a movie?**  
   - Are there specific months or seasons that lead to higher revenues?

Our goal is to convert findings from these questions into **three actionable business recommendations**.

---

## Data Understanding 
An explorative data analysis was done using the following data sets from various sources, to answer our questions.
1. bom.movie_gross.csv.gz
2. im.db.zip
3. tn.movie_budgets.csv.gz
4. tmdb.movies.csv.gz

The data was loaded and previewed and thereafter prepared for analysis by 
- checking the shape, column names and data types. 
- Identify missing values and duplicates.
- Filling in the missing values.........
- Droping rows with missing values...............
- Changing string data type to float to enable mathematical calculations


### **Data Analysis**

In [10]:
# Import the necessary libraries
import pandas as pd
import sqlite3 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

#### Loading the DataSets 

1. Loading the BOM dataset

In [20]:
# Load the compressed BOM dataset
bom_df = pd.read_csv(("zippedData/bom.movie_gross.csv.gz"), compression='gzip')

# Display the first few rows to inspect the data
bom_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


2. Loading the IMDB database

In [39]:
 opening im.db file 

conn = sqlite3.connect('zippedData/im.db/im.db')
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables

SyntaxError: invalid syntax (2664290779.py, line 1)

3. Loading tn.Movie_budgets

In [43]:
df_budget = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
df_budget.head() 
df_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB
